# COVID-19 Data

Tracking COVID-19 using the Johns Hopkins CSSE data hosted in [GitHub](https://github.com/CSSEGISandData/COVID-19)

## Load the Data

Load the CSV data from GitHub using Pandas

In [2]:
import pandas as pd

In [3]:
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

In [4]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,990,990,990,990,990,990,990,990,990,NaN
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,414,414,418,418,422,426,428,428,429,NaN
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,NaN
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,296,296,296,296,296,296,296,296,296,NaN
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,102,119,120,124,124,125,0.0


## Filter Down to Canada

In [5]:
def filter_by_country(df, country):
    is_country = df['Country/Region']==country
    return df[is_country]

In [6]:
canada_confirmed = filter_by_country(confirmed, 'Canada')
canada_deaths = filter_by_country(deaths, 'Canada')
canada_recovered = filter_by_country(recovered, 'Canada')

In [7]:
canada_confirmed

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20
39,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,...,8,9,12,13,21,21,27,32,32,39.0
224,Ontario,Canada,51.2538,-85.3232,0,0,0,0,1,1,...,18,20,20,22,25,28,29,34,36,41.0
225,Alberta,Canada,53.9333,-116.5765,0,0,0,0,0,0,...,0,0,0,0,1,2,4,7,7,19.0
226,Quebec,Canada,52.9399,-73.5491,0,0,0,0,0,0,...,1,1,1,2,2,3,4,4,4,8.0
395,New Brunswick,Canada,46.5653,-66.4619,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


## The last column is the most recent data

In [8]:
canada_confirmed.iloc[:,-1]

39     39.0
224    41.0
225    19.0
226     8.0
395     1.0
Name: 3/11/20, dtype: float64

In [9]:
canada_confirmed.T.iloc[[0,-1]]

,39,224,225,226,395
Province/State,British Columbia,Ontario,Alberta,Quebec,New Brunswick
3/11/20,39,41,19,8,1


In [10]:
top = canada_confirmed.T.head(4)
bottom = canada_confirmed.T.tail(1)
canada_confirmed_latest = pd.concat([top, bottom]).T
canada_confirmed_latest

,Province/State,Country/Region,Lat,Long,3/11/20
39,British Columbia,Canada,49.2827,-123.121,39
224,Ontario,Canada,51.2538,-85.3232,41
225,Alberta,Canada,53.9333,-116.576,19
226,Quebec,Canada,52.9399,-73.5491,8
395,New Brunswick,Canada,46.5653,-66.4619,1


## Map the latest numbers

Ensure that the `ipyleaflet` module is installed with Pip or Conda and make sure the Jupyter extension is also installed.

### Using pip

```
pip install ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet  # can be skipped for notebook 5.3 and above
```

### Using conda

```
conda install -c conda-forge ipyleaflet
```

### JupyterLab extension

If you have JupyterLab, you will also need to install the JupyterLab extension:

```
jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
```

In [11]:
from ipyleaflet import Map, basemaps, CircleMarker

Convert the Pandas `DataFrame` to a `GeoDataFrame` so we can map it.

In [12]:
center = (50.5990364,-93.3975467)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=4)

In [48]:
for row in canada_confirmed_latest.iterrows():
    i, r = row
    marker = CircleMarker()
    marker.location = (r.Lat, r.Long)
    marker.radius = int(r[4])
    marker.color = "red"
    marker.fill_color = "red"
    m.add_layer(marker)
    
display(m)

Map(center=[50.5990364, -93.3975467], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…